# AutoML

O objetivo desse notebook é analisar modelos de regressão linear utilizando o pycaret. Visando encontrar modelos que se encaixem no tipo de dado que será utilizado.

In [ ]:
# Importando as bibliotecas
import pandas as pd
import datetime
from pycaret.regression import *
import matplotlib.pyplot as plt



Esse csv foi obtido a partir do outro notebook, pensando em organização de código é melhor manter pré-processamento separado de treinamento de model.

In [ ]:
# Definindo o csv que será analisado
df = pd.read_csv('df_weekly_fail.csv')
df

,Unnamed: 0,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-2b,basBleedOverPressF-2b,bleedFavTmCmd-1a,bleedFavTmCmd-1b,...,bleedOnStatus-1b,bleedOnStatus-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
0,0,150,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,200,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
2,2,250,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750,1
3,3,300,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
4,4,350,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,1711563,587200,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1711564,1711564,587300,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711565,1711565,587500,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711566,1711566,587700,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875,0


In [ ]:
# Colocando a coluna de data em formato de date_time
df['data_voo'] = pd.to_datetime(df['data_voo'])

In [ ]:
df.columns

Index(['Unnamed: 0', 'recording_time', 'data_voo', 'phaseOfFlight-1',
       'message0418DAA-1', 'message0422DAA-1', 'amscHprsovDrivF-2b',
       'basBleedOverPressF-2b', 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
       'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
       'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
       'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
       'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
       'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
       'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a', 'bleedMonPress-1b',
       'bleedMonPress-2a', 'bleedMonPress-2b', 'bleedOnStatus-1a',
       'bleedOnStatus-1b', 'bleedOnStatus-2b', 'bleedPrecoolDiffPress-1a',
       'bleedPrecoolDiffPress-1b', 'bleedPrecoolDiffPress-2a',
       'bleedPrecoolDiffPress-2b', 'bleedPrsovClPosStatus-1a',
       'bleedPrsovClPosStatus-2a', 'bleedPrsovFbk-1a', 'failure_week'],
      dtype='object')

In [ ]:
# Essas colunas não são relevantes para o modelo
df = df.drop(['recording_time','Unnamed: 0'], axis=1)
df


,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-2b,basBleedOverPressF-2b,bleedFavTmCmd-1a,bleedFavTmCmd-1b,bleedFavTmCmd-2a,bleedFavTmCmd-2b,...,bleedOnStatus-1b,bleedOnStatus-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,failure_week
0,2022-12-06,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
2,2022-12-06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.03125,0.03125,0.03125,0.03125,NaN,NaN,1.750,1
3,2022-12-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1
4,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711563,2023-01-18,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1711564,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.00000,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711565,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.12500,-0.15625,0.06250,0.06250,NaN,NaN,1.875,0
1711566,2023-01-18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.12500,-0.15625,0.03125,0.03125,NaN,NaN,1.875,0


### Auto-ML

Nesta etapa os dados já foram preparados e será utilizando um modelo de regressão linear para tentar prever a semana de falha que houve no modelo.

In [ ]:
# Definindo o setup do pycaret

s = setup(data = df,  target = 'failure_week', fold_strategy = 'timeseries', numeric_features = ['amscHprsovDrivF-2b',
       'basBleedOverPressF-2b', 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
       'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
       'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
       'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
       'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
       'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
       'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a', 'bleedMonPress-1b',
       'bleedMonPress-2a', 'bleedMonPress-2b', 'bleedOnStatus-1a',
       'bleedOnStatus-1b', 'bleedOnStatus-2b', 'bleedPrecoolDiffPress-1a',
       'bleedPrecoolDiffPress-1b', 'bleedPrecoolDiffPress-2a',
       'bleedPrecoolDiffPress-2b', 'bleedPrsovClPosStatus-1a',
       'bleedPrsovClPosStatus-2a', 'bleedPrsovFbk-1a','phaseOfFlight-1'], fold = 3, transform_target = True, train_size=0.3 )

,Description,Value
0,Session id,6440
1,Target,failure_week
2,Target type,Regression
3,Original data shape,"(1711568, 36)"
4,Transformed data shape,"(1711568, 38)"
5,Transformed train set shape,"(513470, 38)"
6,Transformed test set shape,"(1198098, 38)"
7,Numeric features,32
8,Date features,1
9,Rows with missing values,100.0%


In [ ]:
# Comparando os modelos gerados

best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0032,0.0001,0.0096,0.9991,0.0049,0.0290,7.7400
xgboost,Extreme Gradient Boosting,0.0101,0.0009,0.0303,0.9905,0.0156,0.0913,17.9700
dummy,Dummy Regressor,0.1173,0.1084,0.3293,-0.1048,0.2275,0.9911,0.6900


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
# Criando um modelo LightGBM, pois foi o que obteve o melhor resultado
et = create_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0032,0.0001,0.0097,0.9990,0.0049,0.0292
1,0.0032,0.0001,0.0097,0.9990,0.0050,0.0292
2,0.0032,0.0001,0.0095,0.9991,0.0049,0.0287
Mean,0.0032,0.0001,0.0096,0.9991,0.0049,0.0290
Std,0.0000,0.0000,0.0001,0.0000,0.0000,0.0003


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Avaliando o modelo com o pycaret
evaluate_model(et)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…